# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 04**: Data Unions & Joins Pipeline

**Date**: September 23rd 2025

**Student Name**: Ana Carolina Arellano Valdez

**Professor**: Pablo Camarillo Ramirez

# Initializing Spark Session

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on SparkSQL") \
    .master("spark://9e979d8772f7:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/26 02:46:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Build schema for each DataFrame

In [ ]:
# Build schema
# Import your module
from carolinarellano.spark_utils import SparkUtils

schema_agencies = SparkUtils.generate_schema([("agency_id", "int"), ("agency_info", "string")])
schema_brands = SparkUtils.generate_schema([("brand_id", "int"), ("brand_info", "string")])
schema_cars = SparkUtils.generate_schema([("car_id", "int"), ("car_info", "string")])
schema_customers = SparkUtils.generate_schema([("customer_id", "int"), ("customer_info", "string")])
schema_rentals = SparkUtils.generate_schema([("rental_id", "int"), ("rental_info", "string")])

df_agencies = spark.read.schema(schema_agencies).option("header", True).csv("/opt/spark/work-dir/data/car_service/agencies")
df_brands = spark.read.schema(schema_brands).option("header", True).csv("/opt/spark/work-dir/data/car_service/brands")
df_cars = spark.read.schema(schema_cars).option("header", True).csv("/opt/spark/work-dir/data/car_service/cars")
df_customers = spark.read.schema(schema_customers).option("header", True).csv("/opt/spark/work-dir/data/car_service/customers")
df_rentals = spark.read.schema(schema_rentals).option("header", True).csv("/opt/spark/work-dir/data/car_service/rentals")

# Union of DataFrames

In [28]:
from pyspark.sql.functions import get_json_object, col

big_df = df_rentals \
    .withColumn("car_id", get_json_object(col("rental_info"), "$.car_id").cast("int")) \
    .withColumn("customer_id", get_json_object(col("rental_info"), "$.customer_id").cast("int")) \
    .withColumn("agency_id", get_json_object(col("rental_info"), "$.agency_id").cast("int")) \
    .join(df_cars.withColumn("car_name", get_json_object(col("car_info"), "$.car_name")), "car_id", "inner") \
    .join(df_customers.withColumn("customer_name", get_json_object(col("customer_info"), "$.customer_name")), "customer_id", "inner") \
    .join(df_agencies.withColumn("agency_name", get_json_object(col("agency_info"), "$.agency_name")), "agency_id", "inner") \
    .select("rental_id", "car_name", "customer_name", "agency_name") \
    .orderBy("rental_id", ascending=False)


# Show the result of the union

In [29]:
big_df.show(5, truncate=False)

+---------+------------------------------------+-------------+-------------+
|rental_id|car_name                            |customer_name|agency_name  |
+---------+------------------------------------+-------------+-------------+
|17833    |Grimes-Green Model 8                |Jill Sherman |LA Car Rental|
|17832    |Walker, Pratt and Thomas Model 9    |Troy Bell    |Zapopan Auto |
|17831    |Levy Group Model 9                  |Lisa Baldwin |Zapopan Auto |
|17830    |Alvarez-Davis Model 3               |David Walker |LA Car Rental|
|17829    |Patrick, Barrera and Collins Model 6|Blake Jones  |LA Car Rental|
+---------+------------------------------------+-------------+-------------+
only showing top 5 rows


In [ ]:
sc.stop()